In [1]:
# INTEL CORPORATION CONFIDENTIAL AND PROPRIETARY
#
# Copyright © 2020-2021 Intel Corporation.
#
# This software and the related documents are Intel copyrighted
# materials, and your use of them is governed by the express
# license under which they were provided to you (License). Unless
# the License provides otherwise, you may not use, modify, copy,
# publish, distribute, disclose or transmit  this software or the
# related documents without Intel's prior written permission.
# 
# This software and the related documents are provided as is, with
# no express or implied warranties, other than those that are
# expressly stated in the License.

# Input Spike Streamer
This tutorial shows how to use the inputSpikeStreamer, which allows a user to incrementally send spikes to Loihi as they are received from a sensor or other device (file is also possible). To use the inputSpikeStreamer, the user needs to:
 - Specify the wallclock time per Loihi timestep (useful for playing back data in real-time, such as for real-time visualization)
 - Specify the size of packets used to send spikes to Loihi (smaller means more frequent communication, but lower overall bandwidth default of 1024 should be fine for most cases)
 - Specify how many input neurons are required (similar to the number of ports on an NxSDK spike generator)

The user creates the inputSpikeStreamer module with the above parameters, compiles and runs the board in "aSync" mode, then passes lists of spike times (when) and targets (which neuron) to the module. The module will always advance Loihi's timestep to the time of the most recent spike.

In [2]:
import numpy as np
import nxsdk.api.n2a as nx
from nxsdk_modules.spike_streamer.src.streaming import SpikeStreamer

# Create the inputSpikeStreamer

In [3]:
spikesPerPacket = 1024 # larger packets are more efficient, but add latency when very few spikes are present
microsecondsPerTimestep = 1000 # 1ms is typical for sensor interaction, but much smaller timesteps are possible if needed
numInputNeurons = 10 # how many input neurons?
runTime = 10000 #10 seconds if microsecondsPerTimestep is 1000

net = nx.NxNet()
# create the spike streamer
spikeStreamer = SpikeStreamer(net)
spikeStreamer.setupSpikeInput(numInputNeurons, spikesPerPacket, microsecondsPerTimestep)

# Setup your own network and connect the inputSpikeStreamer to it

In [4]:
# replace this with your own network
compProto = nx.CompartmentPrototype()
myNetwork = net.createCompartmentGroup(size=numInputNeurons, prototype=compProto)

# make the connection from spikeStreamer to your network. Modify the connectivity and connection properties here
connProto = nx.ConnectionPrototype()
spikeStreamer.inputLayer.connect(myNetwork, connProto, connectionMask=np.identity(numInputNeurons))

# We'll probe the input neurons to make sure they spike when expected to

In [5]:
# we can probe the input neurons to check that they fire when expected
sProbe = spikeStreamer.inputLayer.probe(nx.ProbeParameter.SPIKE)

# Compile the board and setup the inputSpikeStreamer 
This configures the snips used by the inputSpikeStreamer

In [6]:
# compile the board
compiler = nx.N2Compiler()
board = compiler.compile(net)
spikeStreamer.configureStreamer(board)
board.run(runTime, aSync=True)

INFO:DRV:  SLURM is being run in background
INFO:DRV:  Connecting to 134.134.68.69:36895
INFO:DRV:      Host server up..............Done 0.22s
INFO:DRV:      Encoding axons/synapses.....Done 3.53ms
INFO:DRV:      Compiling Embedded snips....Done 0.17s
INFO:DRV:      Encoding probes.............Done 0.82ms
INFO:HST:  Args chip=0 cpu=0 /home/gorchard/nxsdk-nxsdk/nxsdk/driver/compilers/../../../temp/1583951584.6822233/launcher_chip0_lmt0.bin --chips=1 --remote-relay=0 --epoch=0 
INFO:HST:  Args chip=1 cpu=0 /home/gorchard/nxsdk-nxsdk/nxsdk/driver/../bin/arm/inactive_chips.bin --chips=1 --remote-relay=0 --epoch=0 
INFO:HST:  Args chip=2 cpu=0 /home/gorchard/nxsdk-nxsdk/nxsdk/driver/../bin/arm/inactive_chips.bin --chips=1 --remote-relay=0 --epoch=0 
INFO:HST:  Args chip=3 cpu=0 /home/gorchard/nxsdk-nxsdk/nxsdk/driver/../bin/arm/inactive_chips.bin --chips=1 --remote-relay=0 --epoch=0 
INFO:HST:  Args chip=4 cpu=0 /home/gorchard/nxsdk-nxsdk/nxsdk/driver/../bin/arm/inactive_chips.bin --chips=1

In [7]:
# Send the spike data
"""
# for a real-time demo, this would look something like:
while(True): #some condition to determine when the demo has ended
    spikeTimes, spikeTargets = getSpikesFromSkin()
    spikeStreamer.sendSpikes(spikeTargets, spikeTimes)
"""

# For this artificial test, we'll fake some spike data. 
# The rest of this code block randomly generates spikes and sends them to Loihi in chunks (once per iteration of the for-loop)
maxSpikesPerPacket = 500 # only used for our test/artificial spike generation. In real-time demo the number of spikes isn't limited
numSendIterations = 10 # lets assume we send spikes 10 different times.
minTime = 0

# initialize variables to keep track of all the spike targets and times we randomly generate
allTimes = np.zeros((0,), dtype=int)
allTargets = np.zeros((0,), dtype=int)

for ii in range(numSendIterations):
    # what is the maximum spike time for this iteration?
    # spikes must be sent in chronological order
    maxTime = (ii+1)*runTime//numSendIterations
    
    # initialize a variable to hold the spikes
    spikes = np.zeros((maxSpikesPerPacket,2), dtype=int)
    # randomly generate spike targets (which neuron)
    spikes[:,0] = np.random.randint(0, numInputNeurons, size=(maxSpikesPerPacket,), dtype=int)
    # randomly generate spike times (which neuron)
    spikes[:,1] = np.random.randint(minTime, maxTime, size=(maxSpikesPerPacket,), dtype=int)
    # remove redundant spikes (spike that target the same neuron at the same time)
    spikes = np.unique(spikes, axis=0) # we only want 1 spike per neuron per timestep
    # update the time from which the next chunk of spikes should start
    minTime = maxTime
    
    # sort the the spikes into chronological order (column 1, time, must be increasing)
    spikes = spikes[spikes[:,1].argsort(), :]

    # send the spikes
    spikeStreamer.sendSpikes(spikes[:,0], spikes[:,1])
    
    # record the spikes so we can verify them later against probes
    allTimes = np.concatenate((allTimes, spikes[:,1]))
    allTargets = np.concatenate((allTargets, spikes[:,0]))


INFO:HST:  chip=28 cpu=0 halted, status=0x0
INFO:HST:  chip=29 cpu=0 halted, status=0x0
INFO:HST:  chip=30 cpu=0 halted, status=0x0
INFO:HST:  chip=31 cpu=0 halted, status=0x0


In [8]:
# at any point, "flushSpikes" can be called to flush any buffered spikes to the chip
#spikeInputStreamer.flushSpikes()

In [9]:
# before finishing the run, make sure to advance the chip time to the end of the run
spikeStreamer.advanceTime(runTime)

In [10]:
board.finishRun()
board.disconnect()

INFO:DRV:      Executing...................Done 1.72s
INFO:DRV:      Processing timeseries.......Done 0.04s
INFO:HST:  chip=0 cpu=0 halted, status=0x0


In [11]:
# check that the spikes received from probes match the spikes we injected

# there will be a 1 timestep offset between injecting a spike and seeing it in the spike probe
timeOffset = 1

# spikes that are inject on the last timestep will not appear before the end of the run, so remove them
# (they would appear only if we ran the chip for an extra timestep)
allTargets = allTargets[allTimes+timeOffset<runTime]
allTimes = allTimes[allTimes+timeOffset<runTime]

# check that the data matches exactly
for ii in range(numInputNeurons):
    assert np.sum(np.abs(np.array(sProbe[0][ii].data).nonzero()[0] - (allTimes[allTargets==ii]+timeOffset))) == 0, "error, spike mismatch"
print("Success!!!")

Success!!!
